# Introduction
Loading files into FSLeyes and adjusting settings manually is cumbersom and error prone. Luckily, which FSLeyes it is possible to script the display of files, which saves time and makes the result figures more reproducible. 

There are different ways how we can interact with FSLeyes programatically:
* We can open a Python shell in the FSLeyes GUI from `View` > `Python shell`
* We can open FSLeyes from the command line and run a script at the same time: `fsleyes -r myscript.py`
* for an interactive mode, one can also use a Jupyter Notebook `fsleyes --notebookFile my_notebook.ipynb`

Before we continue, here two links with the relevant User Guides:
* [FSLeyes documentation](https://users.fmrib.ox.ac.uk/~paulmc/fsleyes/userdoc/latest/index.html)
* [FSLeyes Python API documentation](https://users.fmrib.ox.ac.uk/~paulmc/fsleyes/apidoc/latest/)

Please note that I put together this tutorial based on my own understanding of the software and this might or not be the best way to use it. If in doubt, please refer to these User Guides or contact the FSLeyes developers to get advice. 

# Installation
If you installed FSLeyes as part of FSL (prior to version 6), you have a standalone version. If you want to use it via the interactive mode, however, it is recommended to use it as Python package. From FSL version 6 onwards, this is automatically included (I think...).

Here are the steps to install FSLeyes for this tutorial:
* Download and install miniconda (https://docs.conda.io/en/latest/miniconda.html)

* In your terminal create a new environment:

`$ conda create -n fsleyes_tutorial python=3.7`

* When the environment is installed, activate it:

`$ conda activate fsleyes_tutorial`


* Install the FSLeyes Python package to the new environment:

`$ conda install -c conda-forge fsleyes`


* Check that the path to FSLeyes is inside your environment

`$ which fsleyes`


# Start tutorial

If you haven't done so already, load the fsleyes_tutorial conda environment (see above)
Then navigate to the retreat folder:

`cd ~/myPath/tutorial`

Launch the tutorial notebook together with FSLeyes: 

`fsleyes --notebookFile scripts/fsleyes_tutorial.ipynb`

# Basic example
As a first example to see how the interactive mode is working, we will load the human standard MNI brain using the command below. You might notice that you don't need to import the 'load' function, because the some useful packages are already impored when FSLeyes is launched. You might also notice that that the variable `FSLDIR` is acessible within FSLeyes, but only if it was defined within the terminal session, where we launched FSLeyes. 

In [ ]:
import os
load(os.path.expandvars('$FSLDIR/data/standard/MNI152_T1_2mm'))

All files that we load are stored automatically in a list called `overlayList`, which holds all the Image objects. We can access the first element, which is the MNI brain, using regular indexing:

In [ ]:
# first element of list:
overlayList[0]

As a simple manipulation we can change the colour map from greyscale to `Render3`. Note that the default colour maps are directly accessible in this way, but we will other custom colour maps can be included (see more below).

In [ ]:
displayCtx.getOpts(overlayList[0]).cmap = 'Render3'

We can remove this file again, because we don't need it for now:

In [ ]:
overlayList.remove(overlayList[0])

# Goal for this session
The goal of this session is to have a script that automatically creates the display of a structural brain scan together with tractography results from two tracts. The cursor will be centered on the voxel of maximal probability of one tract. The script will contain one variable to define, which subject group we want to display (for example controls, patient group 1, patient group 2, etc. Changing this variable change all the settings adaptively so that a comparable display is created.

# Input files and settings
We need to provide the filenames of interest in an organized way, where Pandas data frames can be handy.
In the tutorial example the filenames have very convenient names, and are organized in a neat way, which will most likely not be the case in a real-life example

In [ ]:
import pandas as pd

# folder where data is stored
mydir = os.path.join('mypath', 'tutorial', 'data')

# filenames
df = pd.DataFrame(columns=['subject_group', 'structural', 'CST', 'MDLF'])
df.loc[len(df)] = ['control', 'structural', 'cst', 'mdlf']
df.loc[len(df)] = ['patient-group1', 'structural', 'cst', 'mdlf']
df.loc[len(df)] = ['patient-group2', 'structural', 'cst', 'mdlf']


Here we define the variable for 'subject-group':

In [ ]:
subject_group = 'control'

In the following lines we define two different colours for the two tracts that we will display. We will later access the correct colour based on the index of the tract. In a similar way, we could define settings that differ for the three subject groups. 

In [ ]:
# colour for the two tracts
my_colours = np.array([[0.  , 0.6 , 1.  ], # blue
                       [1.  , 0.33, 0.68]]) # pink



The tractography results that we load have been normalized with intensities franging from 0 to 1. Therefore, we can apply a comparable threshold of 0.2 to both tracts. In a different situation we might want to have this setting variable depending on tract type or subject group.

In [ ]:
# display range for thresholding the tracts
display_range = (0.2, 1)

# Generate Display

In [ ]:
# import package for colour map (should be at top of script)
from matplotlib.colors import LinearSegmentedColormap

# make sure all previous overlays are removed
overlayList.clear()

# load structural 
structural_fname = f'{df[df.subject_group == subject_group].structural.values[0]}.nii.gz'
load(os.path.join(os.sep, mydir, structural_fname))

# load tractograms 

# loop over hemispheres
for hemi in ['l', 'r']:
    for i_t, tract in enumerate(['CST', 'MDLF']):
        tract_fname = os.path.join(os.sep, mydir, f'{df[df.subject_group == subject_group][tract].values[0]}_{hemi}.nii.gz')
        load(os.path.join(os.sep, mydir, tract_fname))
        
        # set display range and clipping range
        displayCtx.getOpts(overlayList[-1]).clippingRange = display_range
        displayCtx.getOpts(overlayList[-1]).displayRange = display_range
        
        # set colour map specific for tract type
        # use a colour map where luminance linearly increases from black to white
        displayCtx.getOpts(overlayList[-1]).cmap = LinearSegmentedColormap.from_list('mycmap', ['black', my_colours[i_t], 'white'])

        # determine max voxel for MDLF tractogram in left hemisphere
        if (hemi == 'l') & (tract == 'MDLF'):
            max_voxel = np.unravel_index(np.argmax(overlayList[-1].data, axis=None), overlayList[-1].data.shape)

# place cross hair on maximal voxel for MDLF_L
displayCtx = frame.viewPanels[0].displayCtx
displayCtx.location = displayCtx.getOpts(overlayList[-1]).transformCoords(max_voxel, 'voxel', 'display')

# Importing Atlases
It is possible to include custom atlases to FSLeyes and they will be included in the Atlas panel in the GUI. 
Note that any custom atlas files must be described by an XML specification file as outlined [here](https://users.fmrib.ox.ac.uk/~paulmc/fsleyes/userdoc/latest/customising.html#atlases).

It's just a single line of code:

In [ ]:
import fsl
fsl.data.atlases.addAtlas('/myPath/tutorial/myatlas.xml')